In [6]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Load your data
df = pd.read_csv('antihistamine.csv')

# Convert 'Sold_date' to datetime format
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')

# Set 'Sold_date' as the index
df.set_index('Sold_date', inplace=True)

# Aggregate sales data on a weekly basis for each product
weekly_sales = df.groupby([pd.Grouper(freq='W'), 
'Product_details']).agg(total_sold=('Sold_quantity', 'sum')).reset_index()

# Apply one-hot encoding to 'Product_details'
weekly_sales_encoded = pd.concat([weekly_sales, pd.get_dummies(weekly_sales['Product_details'], 
prefix='product')], axis=1).drop('Product_details', axis=1)

# Extract year and week number from 'Sold_date' for temporal features
weekly_sales_encoded['year'] = weekly_sales_encoded['Sold_date'].dt.year
weekly_sales_encoded['week_of_year'] = weekly_sales_encoded['Sold_date'].dt.isocalendar().week

# Normalize the temporal features 
scaler = MinMaxScaler()
features_columns = ['year', 'week_of_year'] + [col for col in weekly_sales_encoded.columns 
if col.startswith('product')]
weekly_sales_encoded[features_columns] = scaler.fit_transform(weekly_sales_encoded[features_columns])

# Split the data into features and target
X = weekly_sales_encoded[features_columns].values
y = weekly_sales_encoded['total_sold'].values

# Adjusting the dataset split into 70% training, 15% validation, and 15% testing
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  # 0.1765 of 85% is roughly 15% of the whole

# Define the model architecture
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(1, activation='relu')    # Output layer
])


# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model with the validation data
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=10)

# Evaluate the model
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE on test set: {rmse}')

# Prepare forecasting features for Feb 1 to 8, 2024, correctly this time
forecast_results = []
for product_col in [col for col in features_columns if col.startswith('product')]:
    forecast_period_features = np.zeros((1, len(features_columns)))
    # Find the index of the year, week_of_year, and current product column
    year_index = features_columns.index('year')
    week_of_year_index = features_columns.index('week_of_year')
    product_index = features_columns.index(product_col)
    
    # Normalize year and week_of_year values
    forecast_period_features[0, year_index] = (2024 - scaler.data_min_[year_index]) / (scaler.data_max_[year_index] - scaler.data_min_[year_index])
    forecast_period_features[0, week_of_year_index] = (5 - scaler.data_min_[week_of_year_index]) / (scaler.data_max_[week_of_year_index] - scaler.data_min_[week_of_year_index])
    forecast_period_features[0, product_index] = 1  # Activate current product

    # Predict and store the result
    predicted_quantity = model.predict(forecast_period_features).flatten()[0]
    product_name = product_col.replace('product_', '')  # Assuming prefix 'product_' is used
    forecast_results.append((product_name, predicted_quantity))

# Display forecasted quantities for each product in a DataFrame
forecast_df = pd.DataFrame(forecast_results, columns=['Product', 'Forecasted Quantity']).sort_values(by='Forecasted Quantity', ascending=False)
print(forecast_df)

Epoch 1/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - loss: 5688.2119 - val_loss: 3813.8250
Epoch 2/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - loss: 2642.8259 - val_loss: 1844.1464
Epoch 3/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 430us/step - loss: 1610.8221 - val_loss: 1715.7489
Epoch 4/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - loss: 1563.5165 - val_loss: 1642.8967
Epoch 5/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - loss: 1671.1611 - val_loss: 1613.4823
Epoch 6/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - loss: 1888.5919 - val_loss: 1661.1140
Epoch 7/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 421us/step - loss: 1744.9266 - val_loss: 1664.4641
Epoch 8/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 429us/step - loss: 1504.1700 - val_loss: 1931.3676
Epoch 9/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 449us/step - loss: 1680.5957 - val_loss: 1746.1428
Epoch 10/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - loss: 1415.5050 - val_loss: 1587.2573
Epoch 11/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 420u

137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - loss: 1478.0326 - val_loss: 1812.3850
Epoch 65/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step - loss: 1326.9517 - val_loss: 1770.5787
Epoch 66/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - loss: 1351.4948 - val_loss: 1771.5922
Epoch 67/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - loss: 1135.3649 - val_loss: 1827.3929
Epoch 68/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - loss: 1186.3257 - val_loss: 1805.0338
Epoch 69/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - loss: 1603.2568 - val_loss: 1941.3977
Epoch 70/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - loss: 1175.4279 - val_loss: 1896.8628
Epoch 71/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 420us/step - loss: 1452.6766 - val_loss: 1780.4344
Epoch 72/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - loss: 1273.5554 - val_loss: 1800.5923
Epoch 73/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 405us/step - loss: 1539.9698 - val_loss: 1887.7502
Epoch 74/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/st

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
                                              Product  Forecasted Quantity
13         Cetirizine (Cetrinova) 10mg Tab 100&#039;s           148.322449
12  Cetirizine (Ceticit/ Cetirigen) 10mg Tab 100&#...            86.157837
19          Cetirizine (Saphzine) 10mg Tab 100&#039;s            49.679008
46                                  Nasatapp 15mg Tab            25.342085
32  Diphenhydramine HCL (Mucomed) 25mg Cap 100&#039;s            16.672047
26  Chlorphenamine Maleate (Riphen/ Allermax) 4mg ...            10.711396
29  Diphenhydramine HCL (BENADRYL AH) 50mg Cap 100...             9.829506
41         Loratadine (Allerta) 10mg Tablet 50&#039;s             7.522806
33   Ebastine Betamethasone ( Co Aleva ) 10/0.5mg Tab             7.251946
11       Cetirizine (CETI-MED) 10mg Tablet 100&#039;s             7.175532
43  Loratadine (Lorarex/ Lorasaph) 10mg Tab 100&#0...             6.934254
38  Levocetirizine Dihydrochloride (Locetin) 5mg 1...          